In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import statsmodels.api as sm
from scipy import stats
import seaborn as sns
import pandas as pd
import numpy as np

In [3]:
# https://archive.ics.uci.edu/ml/datasets/Cervical+cancer+%28Risk+Factors%29
df = pd.read_csv("../../data/risk_factors_cervical_cancer.csv")
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0


In [6]:
df['Biopsy'].mean()

0.0641025641025641

In [10]:
df['Smokes'].value_counts()
df = df.loc[df.Smokes != '?']

In [15]:
def _numeric_transformer(X):
    return pd.DataFrame([X]).T

col_encoders = {
    'Smokes': Pipeline(
                # StandardScaler needs a 2d np.array
                [('wrapper', FunctionTransformer(_numeric_transformer, validate=False, accept_sparse=True))]
    )
}

columns = ['Smokes']

transformers = ColumnTransformer([(col, col_encoders[col], col) for col in columns])

estimator = LogisticRegression(solver='lbfgs')

model = Pipeline([('transformers', transformers), ('estimator', estimator)])
model

Pipeline(memory=None,
     steps=[('transformers', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('Smokes', Pipeline(memory=None,
     steps=[('wrapper', FunctionTransformer(accept_sparse=True, check_inverse=True,
          func=<function _nume...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [16]:
model.fit(df[columns], df['Biopsy'])

Pipeline(memory=None,
     steps=[('transformers', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('Smokes', Pipeline(memory=None,
     steps=[('wrapper', FunctionTransformer(accept_sparse=True, check_inverse=True,
          func=<function _nume...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))])

In [17]:
model.named_steps['estimator'].coef_, model.named_steps['estimator'].intercept_

(array([[0.27341752]]), array([-2.72835949]))